In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [3]:
# import the whole dataset
points_m_from_2017 = pd.read_csv(r"C:\Users\carlo\TrainingsTool\Tennis\Datasets\charting-m-points-from-2017-new.csv", encoding= 'unicode_escape', low_memory=False)

In [4]:
points_m_from_2017.loc[:, ["Pts", "Gm1", "Gm2", "Set1", "Set2", "Serving"]].head(20)

,Pts,Gm1,Gm2,Set1,Set2,Serving
0,0-0,0,0,0,0,MC
1,0-15,0,0,0,0,MC
2,15-15,0,0,0,0,MC
3,15-30,0,0,0,0,MC
4,30-30,0,0,0,0,MC
5,40-30,0,0,0,0,MC
6,40-40,0,0,0,0,MC
7,AD-40,0,0,0,0,MC
8,40-40,0,0,0,0,MC
9,40-AD,0,0,0,0,MC


In [5]:
# only take matches where a specific player played
djokovic_points_from_2017 = points_m_from_2017.loc[points_m_from_2017["match_id"].str.contains("Novak_Djokovic", case = True)]


In [6]:
djokovic_points_from_2017.size

579720

In [7]:
# take the relevant columns from the points where djokovic played
test_df = djokovic_points_from_2017.loc[:, ["Pts", "Gm1", "Gm2", "Set1", "Set2", "Serving", "1st", "2nd"]].head(20)

In [8]:
# ralley colums are being converted into strings
test_df = test_df.astype({"1st": str, "2nd": str})

In [9]:
# join the two ralley colums together into one column with "," seperator
test_df["1st,2nd"] = test_df[["1st", "2nd"]].apply(lambda x: ",".join(x), axis=1)
test_df.head()

,Pts,Gm1,Gm2,Set1,Set2,Serving,1st,2nd,"1st,2nd"
637,0-0,0,0,0,0,ND,4n,4f18f3s2f1f3b3n@,"4n,4f18f3s2f1f3b3n@"
638,0-15,0,0,0,0,ND,4n,6f2n#,"4n,6f2n#"
639,15-15,0,0,0,0,ND,4*,nan,"4*,nan"
640,30-15,0,0,0,0,ND,4n,5b3n@,"4n,5b3n@"
641,40-15,0,0,0,0,ND,4r28f1r2f3b3f2f1f1f3b3b3b1r2n#,nan,"4r28f1r2f3b3f2f1f1f3b3b3b1r2n#,nan"


In [10]:
# drop the two other columns which are irrelevant now
test_df.drop(["1st", "2nd"], axis = 1, inplace=True)
test_df.head()
type(test_df)

pandas.core.frame.DataFrame

In [11]:
# only takes the ralley column and safes it as a dataframe
ralley_df = test_df["1st,2nd"].head(5)
print(ralley_df)
type(ralley_df)

637                   4n,4f18f3s2f1f3b3n@
638                              4n,6f2n#
639                                4*,nan
640                              4n,5b3n@
641    4r28f1r2f3b3f2f1f1f3b3b3b1r2n#,nan
Name: 1st,2nd, dtype: object


pandas.core.series.Series

In [28]:
# creating a dataframe to store the shot_lists in
shot_list_dataframe = pd.DataFrame()
#for i in range(1):
#    shot_list_dataframe["shot_"+str(i)] = np.nan

shot_list_dataframe.head(5)

""


In [29]:
# now comes the tricky bit: each tennis shot has to be its own column to make predictions on that

# go through each character in the "1st,2nd" (ralley) column and follow the schema to add the new columns
# if the next char in the ralley is in the list of possible_shots
# then add the chars until that char as a new shot in a new column

possible_shots = ["f", "b", "r", "s", "v", "z", "o", "p", "y", "l", "m", "h", "i", "j", "k", "t", "u"]
second_serve = ","
len_of_longest_ralley = 0
k = 0
# i is a ralley
for i in ralley_df:
    # Initializeing new shot for each new ralley
    shot = ""
    shot_list = []
    # every char in each ralley will be looked at
    for char in range(0, len(i)):
        # if the char is not in the possible_shots list, the char is appended to the current shot
        if i[char] not in possible_shots and i[char] != second_serve:
            shot = shot + str(i[char])
        # if the char is the ",", shot is done and appended to the shot_list
        elif i[char] == second_serve:
            shot = shot + second_serve
            shot_list.append(shot)
            shot = ""
        # in all other cases, shot is done and appended to the shot_list
        else:
            shot_list.append(shot)
            shot = str(i[char])
    # the last shot has to be appended here, because it is not going into the loop again
    shot_list.append(shot)

    # if the last element of the shot list is "nan" we delete that because we dont want wo predict "nan" and also the "," from the last shot
    if shot_list[-1] == "nan":
        shot_list.pop()
    if shot_list[-1][-1] == "," and len(shot_list) > 1:
        shot_list[-1] = shot_list[-1].rstrip(shot_list[-1][-1])
    
    # finding the longest shot_list/ralley in the dataset
    if len(shot_list) > len_of_longest_ralley:
        len_of_longest_ralley = len(shot_list)

    shot_list_series = pd.Series(shot_list)
    shot_list_small_df = pd.DataFrame(shot_list)

    # now the shot list is beautiful and can add it to the empyt shot_list_dataframe
    #shot_list_dataframe = shot_list_dataframe.append(pd.Series(shot_list, index=shot_list_dataframe.columns[:len(shot_list)]), ignore_index=True)
    #shot_list_dataframe.loc[len(shot_list_dataframe)] = pd.Series(shot_list)
    #pd.concat([shot_list_dataframe, pd.Series([shot_list])], ignore_index=True)
    #df = shot_list_dataframe.append(shot_list_series, ignore_index=True)
    #shot_list_dataframe["ralley_" + str(k)] = pd.Series(shot_list)
    #k = k + 1
    shot_list_dataframe = pd.concat([shot_list_dataframe, shot_list_small_df], ignore_index=True, axis=1)
    shot_list_small_df
    #print(shot_list_small_df)
    #print(shot_list)
#print(type(shot_list_series))    
shot_list_dataframe


,0,1,2,3,4
0,"4n,","4n,","4*,","4n,",4
1,4,6,NaN,5,r28
2,f18,f2n#,NaN,b3n@,f1
3,f3,NaN,NaN,NaN,r2
4,s2,NaN,NaN,NaN,f3
5,f1,NaN,NaN,NaN,b3
6,f3,NaN,NaN,NaN,f2
7,b3n@,NaN,NaN,NaN,f1
8,NaN,NaN,NaN,NaN,f1
9,NaN,NaN,NaN,NaN,f3


In [31]:
# transpose the dataframe from above and concat it to the big dataset with the other stuff
shot_list_df_transposed = shot_list_dataframe.transpose()
shot_list_df_transposed

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,"4n,",4,f18,f3,s2,f1,f3,b3n@,NaN,NaN,NaN,NaN,NaN,NaN
1,"4n,",6,f2n#,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"4*,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"4n,",5,b3n@,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,r28,f1,r2,f3,b3,f2,f1,f1,f3,b3,b3,b1,r2n#


In [45]:
# add the transposed df to the df with the Big one
# shot_list_df_transposed
test_df = test_df.head(5)
test_df.reset_index(drop=True, inplace=True)
df_ralley_taken_apart = pd.concat([test_df, shot_list_df_transposed], ignore_index=False, axis=1, join='outer')
df_ralley_taken_apart.drop(["1st,2nd"], axis = 1, inplace=True)
df_ralley_taken_apart

,Pts,Gm1,Gm2,Set1,Set2,Serving,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0-0,0,0,0,0,ND,"4n,",4,f18,f3,s2,f1,f3,b3n@,NaN,NaN,NaN,NaN,NaN,NaN
1,0-15,0,0,0,0,ND,"4n,",6,f2n#,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15-15,0,0,0,0,ND,"4*,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30-15,0,0,0,0,ND,"4n,",5,b3n@,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40-15,0,0,0,0,ND,4,r28,f1,r2,f3,b3,f2,f1,f1,f3,b3,b3,b1,r2n#


In [ ]:
# now that the dataset is prepared we need to split it up into the dataframes that we want to train models on

# ToDo: do that workflow above for all of djokovics points
# ToDo: first thing would be to split the whole thing into a return dataset and a serve dataset
# ToDo: From those 2 datasets take all the ralleys with the same lengths as seperate Datasets

# note to future self: start with a set for 2 or 3 shots and look at the accuracy of that
